<a href="https://colab.research.google.com/github/steveding1/aws-dataengineering/blob/main/s3_df_rds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install fsspec
!pip install s3fs
!pip install boto3

In [35]:
!pip install psycopg2==2.9.3

     |████████████████████████████████| 380 kB 7.4 MB/s 
  Created wheel for psycopg2: filename=psycopg2-2.9.3-cp37-cp37m-linux_x86_64.whl size=471444 sha256=653d1cb92643d82d07488052d6689823d7477b616fe8072b05cd56886aa68b96
  Stored in directory: /root/.cache/pip/wheels/20/78/2c/d2f59d80d97357ffd6526f209083e46d57827d94d89ac8c91a
Successfully built psycopg2
  Attempting uninstall: psycopg2
    Found existing installation: psycopg2 2.7.6.1
    Uninstalling psycopg2-2.7.6.1:
      Successfully uninstalled psycopg2-2.7.6.1


In [1]:
import pandas as pd
from sqlalchemy import create_engine
import datetime
import configparser
import boto3

In [2]:
def s3_to_df(access_key,secret_key,bucket_url)->pd.DataFrame:
  aws_credentials = {"key":access_key,"secret":secret_key}
  return pd.read_csv(bucket_url,
                     storage_options=aws_credentials,
                     compression='gzip')

In [ ]:
s3_to_df(access_key,secret_key,'s3://sding-airbnb/australia_barwon-south-west-vic.listings.csv.gz')[:3]

In [4]:
#def main():
parser = configparser.ConfigParser()
parser.read('conf.conf')
sql_host = parser.get("postgres","host")
sql_user = parser.get("postgres","username")
sql_password = parser.get("postgres","password")
sql_db = parser.get("postgres","database")
sql_table = parser.get("postgres","table")
access_key = parser.get("boto_config","access_key")
secret_key = parser.get("boto_config","secret_key")
bucket_name = parser.get("boto_config","bucket_name")

connection_string = "postgresql+psycopg2://{uid}:{pwd}@{host}:{port}/{db}".format(
        uid=sql_user,
        pwd=sql_password,
        host=sql_host,
        port= parser.get("postgres","port"),
        db=sql_db
    )
engine = create_engine(connection_string)

session = boto3.Session(aws_access_key_id=access_key,aws_secret_access_key=secret_key)
resource = session.resource('s3')
bucket = resource.Bucket(bucket_name)
for obj in bucket.objects.all():
  data_df=s3_to_df(access_key,secret_key,'s3://'+bucket_name+'/'+obj.key)
  data_df.to_sql(sql_table, con=engine, index=False, if_exists="append")

KeyboardInterrupt: ignored